In [93]:
import numpy as np
import cv2
import scipy

In [94]:
import scipy.ndimage

In [224]:
def cut_and_save(image,part_number,angle,rootdir,file_name,start,end):
    cut_image = image[:70,start:end]
    cut_image = scipy.ndimage.interpolation.rotate(cut_image,angle, reshape=True,cval=255)
    cv2.imwrite(os.path.join(rootdir, file_name +".jpg") ,cut_image)

In [65]:
import os
rootdir = "/Users/Indix/Downloads/amazon_captcha/processed/"
first_rootdir = "/Users/Indix/Downloads/amazon_captcha/first_char"

for subdir, dirs, files in os.walk(rootdir):
    for file in files:        
        if not (file == ".DS_Store"):
            image = cv2.imread(os.path.join(subdir, file))
#             cut_and_save(image,1,13,"/Users/Indix/Downloads/amazon_captcha/first_char",file,0,27)
#             cut_and_save(image,2,-13,"/Users/Indix/Downloads/amazon_captcha/second_char",file,26,56)
#             cut_and_save(image,3,13,"/Users/Indix/Downloads/amazon_captcha/third_char",file,55,84)
#             cut_and_save(image,4,-13,"/Users/Indix/Downloads/amazon_captcha/fourth_char",file,82,111)
#             cut_and_save(image,5,13,"/Users/Indix/Downloads/amazon_captcha/fifth_char",file,112,138)

#             cut_and_save(image,6,-13,"/Users/Indix/Downloads/amazon_captcha/sixth_char",file,139,190)

NameError: name 'cut_and_save' is not defined

In [298]:
def high_low(x):
    return x if x == 17850 else 0

pic_hash = {}
def find_image_cuts(file_name,image):
    print file_name
    pic_hash[file_name] = image
    columns_summed = image.sum(axis=0)
    sums = [item[0] for item in columns_summed]
    final_sums =  np.vectorize(high_low)(sums)
    l = np.array_split(final_sums,5);
    array_size = []
    splits = []
#     print l
    for ix,i in enumerate(l) :
        print ix , i
        if ix == 0:
            array_size.append(0)
            split_points = np.where(i==17850)[0]
            if(split_points.size == 0):
                split_points = np.where(l[ix+1]==17850)[0]
                l[ix+1][split_points[0]] = 0
                splits.append(len(i) + split_points[0])
            else :                
                splits.append(split_points[0])
        else:
            print len(i)
            print splits
            value = array_size[ix-1] + len(i)
            array_size.append(value)            
            split_points = np.where(i==17850)[0]
            print ix , i
            print "initial ", split_points
            if(split_points.size == 0):
                print "zero case "
                previous_points_reverse = l[ix-1][::-1]
                split_points = np.where(previous_points_reverse==17850)[0]            
                print "final" , split_points
            splits.append( array_size[ix] + split_points[0])
    print splits        
    return splits

In [299]:
import os
rootdir = "/Users/Indix/Downloads/amazon_captcha/actual_images1/"
altered_rootdir = "/Users/Indix/Downloads/amazon_captcha/processed"

def f(x):
    return 0 if x < 15  else 255

def find_first_non_white_part(image):
    fnwc = 0
    for index, x in np.ndenumerate(image):
        if fnwc == 0:
            if x < 17750:
                fnwc = index[0] +1
    return fnwc            
        

for subdir, dirs, files in os.walk(rootdir):
    for file in files:        
        if not (file == ".DS_Store"):
            image = cv2.imread(os.path.join(subdir, file))
            processer = np.vectorize(f)
            processed_image = processer(image)
            columns_summed = processed_image.sum(axis=0)
            sums = [item[0] for item in columns_summed]
            first_char = 10 + find_first_non_white_part(sums[10:])
            last_char = len(sums) - find_first_non_white_part(sums[::-1])
            final_processed_image = processed_image[:,first_char:last_char,]
            cv2.imwrite(os.path.join(altered_rootdir, file +".jpg") ,processed_image[:,first_char:last_char,])                               
            splits = find_image_cuts(file,final_processed_image)
            cut_and_save(final_processed_image,1,13,"/Users/Indix/Downloads/amazon_captcha/first_char",file,0,splits[0])
            cut_and_save(final_processed_image,2,-13,"/Users/Indix/Downloads/amazon_captcha/second_char",file,splits[0],splits[1])
            cut_and_save(final_processed_image,3,13,"/Users/Indix/Downloads/amazon_captcha/third_char",file,splits[1],splits[2])
            cut_and_save(final_processed_image,4,-13,"/Users/Indix/Downloads/amazon_captcha/fourth_char",file,splits[2],splits[3])
            cut_and_save(final_processed_image,5,13,"/Users/Indix/Downloads/amazon_captcha/fifth_char",file,splits[3],splits[4])
            cut_and_save(final_processed_image,6,-13,"/Users/Indix/Downloads/amazon_captcha/sixth_char",file,splits[4],len(final_processed_image[0]))
            

95
0 [    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0 17850]
1 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
33
[32]
1 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
initial  []
zero case 
final [0]
2 [17850     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0 17850     0     0     0     0     0     0     0]
33
[32, 33]
2 [17850     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0 17850     0     0     0     0     0     0     0]
initial  [ 0 25]
3 [    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0 17850 17850     0     0     0     0     0     0
     0     0     0     0   

In [122]:
from math import ceil

def takespread(sequence, num):
    length = float(len(sequence))
    for i in range(num):
        yield sequence[int(ceil(i * length / num))]

In [34]:
import os
rootdir = "/Users/Indix/Downloads/amazon_captcha/processed/"
first_rootdir = "/Users/Indix/Downloads/amazon_captcha/first_char"

np_sums = None 
for subdir, dirs, files in os.walk(rootdir):
    for file in files:        
        if not (file == ".DS_Store"):
            image = cv2.imread(os.path.join(subdir, file))
            columns_summed = image.sum(axis=0)
            sums = [item[0] for item in columns_summed]
            np_sums = np.array(sums)
            possible_spilt_points = np_sums.argsort()[-15:][::-1]
            possible_spilt_points.sort()
#             print file , possible_spilt_points
#             cut_and_save(cv2.imread(os.path.join(subdir, file)),1,13,"/Users/Indix/Downloads/amazon_captcha/first_char",file,0,best_splits[0])
#             cut_and_save(cv2.imread(os.path.join(subdir, file)),2,-13,"/Users/Indix/Downloads/amazon_captcha/second_char",file,best_splits[0] +1 ,best_splits[1])
#             cut_and_save(cv2.imread(os.path.join(subdir, file)),3,13,"/Users/Indix/Downloads/amazon_captcha/third_char",file,best_splits[1] +1 ,best_splits[2])
#             cut_and_save(cv2.imread(os.path.join(subdir, file)),4,-13,"/Users/Indix/Downloads/amazon_captcha/fourth_char",file,best_splits[2] +1 ,best_splits[3])
#             cut_and_save(cv2.imread(os.path.join(subdir, file)),5,13,"/Users/Indix/Downloads/amazon_captcha/fifth_char",file,best_splits[3] +1 ,best_splits[4])
#             cut_and_save(cv2.imread(os.path.join(subdir, file)),6,-13,"/Users/Indix/Downloads/amazon_captcha/sixth_char",file,best_splits[4] +1 ,image.shape[1])            

In [37]:
import os
rootdir = "/Users/Indix/Downloads/amazon_captcha/processed/"
first_rootdir = "/Users/Indix/Downloads/amazon_captcha/first_char"

np_sums = None 
for subdir, dirs, files in os.walk(rootdir):
    for file in files:        
        if not (file == ".DS_Store"):
            image = cv2.imread(os.path.join(subdir, file))
            columns_summed = image.sum(axis=0)
            sums = [item[0] for item in columns_summed]
            np_sums = np.array(sums)
            possible_spilt_points = np_sums.argsort()[-15:][::-1]
            possible_spilt_points
            print file 
            print possible_spilt_points
            print sums            
#             cut_and_save(cv2.imread(os.path.join(subdir, file)),1,13,"/Users/Indix/Downloads/amazon_captcha/first_char",file,0,best_splits[0])
#             cut_and_save(cv2.imread(os.path.join(subdir, file)),2,-13,"/Users/Indix/Downloads/amazon_captcha/second_char",file,best_splits[0] +1 ,best_splits[1])
#             cut_and_save(cv2.imread(os.path.join(subdir, file)),3,13,"/Users/Indix/Downloads/amazon_captcha/third_char",file,best_splits[1] +1 ,best_splits[2])
#             cut_and_save(cv2.imread(os.path.join(subdir, file)),4,-13,"/Users/Indix/Downloads/amazon_captcha/fourth_char",file,best_splits[2] +1 ,best_splits[3])
#             cut_and_save(cv2.imread(os.path.join(subdir, file)),5,13,"/Users/Indix/Downloads/amazon_captcha/fifth_char",file,best_splits[3] +1 ,best_splits[4])
#             cut_and_save(cv2.imread(os.path.join(subdir, file)),6,-13,"/Users/Indix/Downloads/amazon_captcha/sixth_char",file,best_splits[4] +1 ,image.shape[1])            

10.jpg
[ 76  77  52 101  24 100  51  53 125  39  38  37  23  75  33]
[15799, 15036, 13767, 14279, 14276, 13516, 14015, 15544, 16056, 15800, 15546, 16049, 16053, 16059, 16051, 15806, 16311, 15294, 13769, 14276, 14532, 15291, 16303, 17329, 17833, 16569, 15553, 15038, 14793, 14787, 15290, 16056, 16568, 17327, 17084, 17326, 17077, 17330, 17582, 17586, 17311, 17310, 17063, 16298, 14778, 14273, 14027, 14027, 13773, 14782, 16301, 17822, 17834, 17820, 17066, 15800, 15546, 14260, 13259, 12495, 14278, 16048, 17054, 16815, 17081, 17069, 16830, 17068, 16829, 16825, 16824, 17082, 16824, 16827, 17325, 17327, 17836, 17835, 17325, 16573, 15803, 14766, 14017, 14266, 14274, 13511, 14023, 14778, 16054, 16294, 16312, 16301, 16557, 16312, 16801, 16818, 16049, 14778, 15801, 16553, 17831, 17833, 17056, 16818, 16310, 16558, 16316, 16561, 16565, 16565, 16310, 15542, 15794, 15800, 16047, 15805, 16054, 16555, 15795, 15536, 12495, 10219, 11734, 14020, 17322, 17818, 17058, 17318, 17326, 17070, 17070, 17324, 17079,

In [201]:
to_be_split_image = cv2.imread("/Users/Indix/Downloads/amazon_captcha/actual_images1/108.jpg")

In [1]:
import Image



In [13]:
from tesseract import image_to_string

ImportError: No module named tesseract

In [ ]:

print image_to_string(Image.open('test.png'))
print image_to_string(Image.open('test-english.jpg'), lang='eng')

In [14]:
import os 

In [16]:
os.environ

{'LC_CTYPE': 'UTF-8', 'TERM_PROGRAM_VERSION': '3.0.13', 'LOGNAME': 'raghavan', 'USER': 'raghavan', 'PATH': '/Users/Indix/anaconda2/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Users/Indix/anaconda2/bin', 'HOME': '/Users/Indix', 'TERM_PROGRAM': 'iTerm.app', 'TERM': 'xterm-color', 'Apple_PubSub_Socket_Render': '/private/tmp/com.apple.launchd.Zmua5KlukB/Render', 'COLORFGBG': '12;8', 'SHLVL': '1', 'SECURITYSESSIONID': '186a8', 'XPC_FLAGS': '0x0', 'ITERM_SESSION_ID': 'w0t4p0:B8A878B2-79E0-44E7-92B2-F497171843FB', 'TERM_SESSION_ID': 'w0t4p0:B8A878B2-79E0-44E7-92B2-F497171843FB', 'XPC_SERVICE_NAME': '0', 'JPY_PARENT_PID': '20820', 'OMF_PATH': '/Users/Indix/.local/share/omf', 'SSH_AUTH_SOCK': '/private/tmp/com.apple.launchd.pfZfODbOTW/Listeners', 'SHELL': '/usr/local/bin/fish', 'GIT_PAGER': 'cat', 'ITERM_PROFILE': 'Default', 'OMF_CONFIG': '/Users/Indix/.config/omf', 'TMPDIR': '/var/folders/_w/d78f4hrn7mxd7xpjtqf7fff40000gn/T/', 'MPLBACKEND': 'module://ipykernel.pylab.backend_inline', 'CLI

In [63]:
image = cv2.imread(os.path.join("/Users/Indix/Downloads/amazon_captcha/processed/", "10.jpg"))

In [90]:
print image[:][25]

[[255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [254 254 254]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [ 91  91  91]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [ 43  43  43]
 [199 199 199]
 [245 245 245]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [103 103 103]
 [ 20  20  20]
 [  0   0   0]
 [  0   0   0]
 [ 58  58  58]
 [231 231 231]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 

In [54]:
def f(x):

    return 0 if x < 15  else 255

In [89]:
processer = np.vectorize(f)
processed_image = processer(image)
print processed_image[:][23]

[[255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [  0   0   0]
 [  0   0   0]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [  0   0   0]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 255]
 [255 255 

In [58]:
processer =

array([255,  22, 141])

In [ ]:
def load_image( infilename ) :
    img = Image.open( infilename )
    img.load()
    data = np.asarray( img, dtype="int32" )
    return data

In [ ]:
image_array = load_image("/Users/Indix/Documents/3.jpg")

In [ ]:
print [1,2,3][::-1]